## Imports

In [1]:
__author__ = "Jordan Allen"

from typing import List

import numpy as np
import pandas as pd

from tqdm import tqdm

from scipy.stats import beta, gamma
from scipy.stats import chi2_contingency
from scipy.special import betaln

# import util functions
from utils.hypothesis_test import Hypothesis_AB_Test
from utils.bayesian_test import Bayesian_AB_Test

# import visualisation / plotting
import matplotlib
matplotlib.use("Agg")

from utils.graph import visualisation # conda install -n python3 -c conda-forge colorlover
import plotly.graph_objects as go
# Init visualisation tool
plot = visualisation(renderer="vscode") # vscode | iframe for browsers

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.width', 10000)

tqdm.pandas()

## Define parameters

In [2]:
# Define Campaign Parameters
# Consider how many variants you have in your test

# ---------------------------------------------------------
# Number of interactions (opens/clicks/conversions etc..)
n_ints_a = 60
n_ints_b = 44
n_ints_c = 22
#n_ints_d = 147
#n_ints_e = 138
#n_ints_f = 139
#n_ints_g = 132

# ---------------------------------------------------------
# Number of impressions (sends/deliveries etc..)
N_IMPR_A = 37971
N_IMPR_B = 25651
N_IMPR_C = 13411
#N_IMPR_D = 5476
#N_IMPR_E = 5552
#N_IMPR_F = 5436
#N_IMPR_G = 5400

# ---------------------------------------------------------
# Interaction rate (open-rate/click-rate/conversion-rate etc..)
INTR_A = n_ints_a/N_IMPR_A
INTR_B = n_ints_b/N_IMPR_B
INTR_C = n_ints_c/N_IMPR_C
#INTR_D = n_ints_d/N_IMPR_D
#INTR_E = n_ints_e/N_IMPR_E
#INTR_F = n_ints_f/N_IMPR_F
#INTR_G = n_ints_g/N_IMPR_G

# ---------------------------------------------------------
# NOTE: All of the above example metrics are beta distributed. If you wish to analyse a gamma distributed variable (e.g., revenue per send) then input it here
RpS_A = 18.15
RpS_B = 18.01
RpS_C = 16.93

# ---------------------------------------------------------
# Config for evaluation
config ={'metrics':
            {'ks': True, # calc. Kolmorogorov-Smirnov
             'ws': True, # calc. Wasserstein
             'jsd': False, # calc. Jensen-Shannon-Divergence
             'n_samples': 1000} # number of samples
         }

# Plotting
WIDTH_SAVE, HEIGHT_SAVE = 1200, 400

In [3]:
# Edit this part depending on the number of variants you have

variant_names = ['A', 'B', 'C']
INTR_list = [INTR_A, INTR_B, INTR_C]
IMPR_list = [N_IMPR_A, N_IMPR_B, N_IMPR_C]
ints_list = [n_ints_a, n_ints_b, n_ints_c]
rps_list = [RpS_A, RpS_B, RpS_C]

## Campaign overview

In [4]:
# Edit this section, depending on how many variants you have
print(f'Observations:')
for i in range(len(variant_names)):
    print(f'Impressions {variant_names[i]}: {IMPR_list[i]} - interactions {variant_names[i]}: {ints_list[i]}')

print(f'Average statistics (without uncertainty):')
for i in range(len(variant_names)):
    print(f'Interaction-Rate (INTR) {variant_names[i]}: {100*INTR_list[i]:.3f}%')
    print(f'Revenue-per-Send (RpS) {variant_names[i]}: {rps_list[i]:.2f}')

Observations:
Impressions A: 37971 - interactions A: 60
Impressions B: 25651 - interactions B: 44
Impressions C: 13411 - interactions C: 22
Average statistics (without uncertainty):
Interaction-Rate (INTR) A: 0.158%
Revenue-per-Send (RpS) A: 18.15
Interaction-Rate (INTR) B: 0.172%
Revenue-per-Send (RpS) B: 18.01
Interaction-Rate (INTR) C: 0.164%
Revenue-per-Send (RpS) C: 16.93


## Classical hypothesis testing

In [5]:
hypo = Hypothesis_AB_Test()

In [6]:
print('INTR')
n_samples_required = hypo.calc_sample_size(INTR_list)
print(f'Required sample size: {n_samples_required}')

hypo.chi2_test(ints_list, IMPR_list,variant_names, print_opt=True)

INTR
Required sample size: 1977905
Chi-square test for Variant A vs Variant B:
Chi-square statistic: 0.09859231459550383
P-value: 0.7535254959454891

Chi-square test for Variant A vs Variant C:
Chi-square statistic: 0.0006023035118997319
P-value: 0.9804203842382876

Chi-square test for Variant B vs Variant C:
Chi-square statistic: 0.0017129335582071314
P-value: 0.9669868968960581



{}

## Bayesian testing

In [7]:
bayes = Bayesian_AB_Test()

In [8]:
# Interaction-Rate (INTR) - Beta distribution
# Calc. probabilities and losses
print(f'Performance metric: Interaction-Rate (INTR)')
P = bayes.p_ab(ints_list, IMPR_list, dist='beta')
for i in range(len(variant_names)):
    print(f'Probability of winner {variant_names[i]}: {P[i]*100:.2f}%(sampled)')

# Attr-revenue-per-Send (ARpS) - Gamma distribution
# Calc. probabilities and losses
print(f'Performance metric: Revenue-per-Send (RpS)')
P = bayes.p_ab(rps_list, IMPR_list, dist='gamma')
for i in range(len(variant_names)):
    print(f'Probability of winner {variant_names[i]}: {P[i]*100:.2f}%(sampled)')

Performance metric: Interaction-Rate (INTR)
Probability of winner A: 20.53%(sampled)
Probability of winner B: 44.73%(sampled)
Probability of winner C: 34.74%(sampled)
Performance metric: Revenue-per-Send (RpS)
Probability of winner A: 100.00%(sampled)
Probability of winner B: 0.00%(sampled)
Probability of winner C: 0.00%(sampled)


## PDF plots

In [9]:
# These plots show the probability denisty function for each of the variables
# Note the overlap (or lack or overlap) between each of the distributions
# This is why we do or do not have 100% Bayesian probability that 1 variant is best
# We will see 100% probability if there is no overlap with the top variant, otherwise we will see less than 100%

x1 = np.linspace(0, max(INTR_list)*1.5, 1001)
p_data = []
for i in range(len(variant_names)):
    p_data.append(plot.plot(x=x1, y=beta.pdf(x1, ints_list[i], IMPR_list[i]-ints_list[i]), color=i, opacity=0.7, name=variant_names[i], showlegend=True))
layout = plot.layout(title=f'', x_label='INTR', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()

x2 = np.linspace(min(rps_list)*0.8, max(rps_list)*1.2, 1001)
p_data = []
for i in range(len(variant_names)):
    p_data.append(plot.plot(x=x2, y=gamma.pdf(x2, a=rps_list[i]*IMPR_list[i], scale=1/IMPR_list[i]), color=i, opacity=0.7, name=variant_names[i], showlegend=True))
layout = plot.layout(title=f'', x_label='RpS', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()

## Real example (time-series data)

In [25]:
# Read in your dataset
df = pd.read_csv('example_data/example_data.csv')

# Make sure it is formatted correctly
df['sendDate'] = pd.to_datetime(df['sendDate'])
df.sort_values(by='sendDate', inplace=True)
df.rename({'sendDate':'t'}, axis=1,inplace=True)
df.drop ('conversions', axis=1, inplace=True) # Switch this with clicks if you wish to find conversion rate (remember to change all the subsequent code blocks as well)
df_pivoted = df.groupby(['t', 'exp_group']).sum().unstack('exp_group').reset_index()
df_pivoted.columns = [f'{col[0]}_{col[1]}' if col[1] else col[0] for col in df_pivoted.columns]
df_pivoted.reset_index(drop=True,inplace=True)
df_pivoted['t'] = (df_pivoted['t'] - df_pivoted['t'].min()).dt.days
df_pivoted.reset_index(drop=True,inplace=True)


In [26]:
# Find chi2 and pvalue for click rate for every day of the test by looking at the cumulative numbers of impressions and clicks
for group in df.exp_group.unique():
    df_pivoted[f'cum_impressions_{group}'] = df_pivoted[f'impressions_{group}'].cumsum()
    df_pivoted[f'cum_clicks_{group}'] = df_pivoted[f'clicks_{group}'].cumsum()
    df_pivoted[f'cum_revenue_{group}'] = df_pivoted[f'attr_revenue_eur_{group}'].cumsum()

df_pivoted['test'] = df_pivoted.progress_apply(lambda x: hypo.chi2_test([x[f'cum_clicks_{group}'] for group in df.exp_group.unique()], [x[f'cum_impressions_{group}'] for group in df.exp_group.unique()], df.exp_group.unique(), print_opt=False), axis=1)

keys = df_pivoted['test'].iloc[0].keys()
df[list(keys)] = df_pivoted['test'].apply(lambda x: pd.Series(x))
df2 = df[list(keys)].dropna()

for var1 in keys:
    df_pivoted[[f'chi2_ctr_{var1}', f'pvalue_ctr_{var1}']] = df2[var1].apply(lambda x: pd.Series(x))

df_pivoted.drop(columns=['test'], inplace=True)

 44%|████▍     | 40/91 [00:00<00:00, 394.80it/s]

100%|██████████| 91/91 [00:00<00:00, 443.17it/s]


In [27]:
# Calculate the Bayesian probability that each of the variants is the best at each successive day
P_pivoted = df_pivoted.progress_apply(lambda x: bayes.p_ab([x['cum_clicks_A'], x['cum_clicks_B'], x['cum_clicks_C']], [x['cum_impressions_A'], x['cum_impressions_B'], x['cum_impressions_C']], dist='beta'), axis=1)
df_pivoted['P_A_b'] = [_[0] for _ in P_pivoted]
df_pivoted['P_B_b'] = [_[1] for _ in P_pivoted]
df_pivoted['P_C_b'] = [_[2] for _ in P_pivoted]

P_pivoted_arps = df_pivoted.progress_apply(lambda x: bayes.p_ab([x['cum_revenue_A'], x['cum_revenue_B'], x['cum_revenue_C']], [x['cum_impressions_A'], x['cum_impressions_B'], x['cum_impressions_C']], dist='gamma'), axis=1)
df_pivoted['P_A_b_arps'] = [_[0] for _ in P_pivoted_arps]
df_pivoted['P_B_b_arps'] = [_[1] for _ in P_pivoted_arps]
df_pivoted['P_C_b_arps'] = [_[2] for _ in P_pivoted_arps]

100%|██████████| 91/91 [00:01<00:00, 51.13it/s]


In [28]:
df_pivoted[0:5]

,t,impressions_A,impressions_B,impressions_C,clicks_A,clicks_B,clicks_C,attr_revenue_eur_A,attr_revenue_eur_B,attr_revenue_eur_C,cum_impressions_A,cum_clicks_A,cum_revenue_A,cum_impressions_C,cum_clicks_C,cum_revenue_C,cum_impressions_B,cum_clicks_B,cum_revenue_B,chi2_ctr_A_C,pvalue_ctr_A_C,chi2_ctr_A_B,pvalue_ctr_A_B,chi2_ctr_C_B,pvalue_ctr_C_B,P_A_b,P_B_b,P_C_b,P_A_b_arps,P_B_b_arps,P_C_b_arps
0,0,258354,304860,220423,7390,14880,16305,1.839423e+06,5.081063e+06,3.503059e+06,258354,7390,1.839423e+06,220423,16305,3.503059e+06,304860,14880,5.081063e+06,5203.114196,0.0,1502.747354,0.000000e+00,1449.944520,0.000000e+00,0.0,0.0,1.0,0.0,1.0,0.0
1,1,464195,418846,334702,23162,18329,21634,7.840881e+06,5.494572e+06,7.411816e+06,722549,30552,9.680304e+06,555125,37939,1.091488e+07,723706,33209,1.057564e+07,4201.773383,0.0,111.336959,4.992178e-26,3014.698858,0.000000e+00,0.0,0.0,1.0,0.0,1.0,0.0
2,2,287594,302063,487774,18014,18703,30785,6.419959e+06,5.204664e+06,1.458799e+07,1010143,48566,1.610026e+07,1042899,68724,2.550286e+07,1025769,51912,1.578030e+07,3024.257106,0.0,69.356818,8.216810e-17,2201.019813,0.000000e+00,0.0,0.0,1.0,0.0,0.0,1.0
3,3,426412,473500,363760,22487,33619,18370,6.259987e+06,1.221433e+07,7.675875e+06,1436555,71053,2.236025e+07,1406659,87094,3.317874e+07,1499269,85531,2.799463e+07,2098.601638,0.0,836.391798,6.607638e-184,307.586844,7.326912e-69,0.0,0.0,1.0,0.0,0.0,1.0
4,4,296262,336264,230829,9582,18207,7408,2.164983e+06,5.677361e+06,1.168301e+06,1732817,80635,2.452523e+07,1637488,94502,3.434704e+07,1835533,103738,3.367199e+07,2134.876271,0.0,1812.539340,0.000000e+00,22.940352,1.671068e-06,0.0,0.0,1.0,0.0,1.0,0.0


In [29]:
# Now we show the p-value of the click rate throughout the experiment
# The first plot shows the p-value that there is a significant different between variants A and B, the second between A and C, and the third between B and C
df = df_pivoted
# CHI2 - CTR
p_data = [ plot.plot(x=df.t, y=df.pvalue_ctr_A_B, color=1, opacity=0.7, name='p-value', showlegend=True),
           plot.plot(x=list(range(df.t.iloc[-1]+1)), y=(df.t.iloc[-1]+1)*[0.05], color=0, opacity=1, fill=None, linewidth=3, name='<0.05', showlegend=True),
           ]
layout = plot.layout(title=f'Chi2-test A/B - p-value (CTR)', x_label='# days', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()

p_data = [ plot.plot(x=df.t, y=df.pvalue_ctr_A_C, color=1, opacity=0.7, name='p-value', showlegend=True),
           plot.plot(x=list(range(df.t.iloc[-1]+1)), y=(df.t.iloc[-1]+1)*[0.05], color=0, opacity=1, fill=None, linewidth=3, name='<0.05', showlegend=True),
           ]
layout = plot.layout(title=f'Chi2-test A/C - p-value (CTR)', x_label='# days', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()

p_data = [ plot.plot(x=df.t, y=df.pvalue_ctr_C_B, color=1, opacity=0.7, name='p-value', showlegend=True),
           plot.plot(x=list(range(df.t.iloc[-1]+1)), y=(df.t.iloc[-1]+1)*[0.05], color=0, opacity=1, fill=None, linewidth=3, name='<0.05', showlegend=True),
           ]
layout = plot.layout(title=f'Chi2-test B/C - p-value (CTR)', x_label='# days', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()

In [30]:
# Now we show the Bayesian probability of each variant being the best for click rate and revenue-per-send throughout the experiment
# There is a red line at 95%, to show easily when we a variant has above 95% probability of being the best, this can be changed to any desired % on line 4
# BAYES - CTR
p_data = [ plot.plot(x=list(range(df.t.iloc[-1]+1)), y=(df.t.iloc[-1]+1)*[0.95], color=0, opacity=1, fill=None, linewidth=3, name='>0.95', showlegend=True),
           plot.plot(x=df.t, y=df['P_A_b'], color=0, opacity=0.7, name='P(A>BnC)', showlegend=True),
           plot.plot(x=df.t, y=df['P_B_b'], color=1, opacity=0.7, name='P(B>AnC)', showlegend=True),
           plot.plot(x=df.t, y=df['P_C_b'], color=2, opacity=0.7, name='P(C>AnB)', showlegend=True),
           ]
layout = plot.layout(title=f'Probability of variants being better than the rest - CTR', x_label='# days', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()

# BAYES - ARpS
p_data_3 = [ plot.plot(x=list(range(df.t.iloc[-1]+1)), y=(df.t.iloc[-1]+1)*[0.95], color=0, opacity=1, fill=None, linewidth=3, name='>0.95', showlegend=True),
           plot.plot(x=df.t, y=df['P_A_b_arps'], color=0, opacity=0.7, name='P(A>BnC)', showlegend=True),
           plot.plot(x=df.t, y=df['P_B_b_arps'], color=1, opacity=0.7, name='P(B>AnC)', showlegend=True),
           plot.plot(x=df.t, y=df['P_C_b_arps'], color=2, opacity=0.7, name='P(C>AnB)', showlegend=True),
           ]
layout_3 = plot.layout(title=f'Probability of variants being better than the rest - ARpS', x_label='# days', y_label='p', theme='dark', width=1200, height=400)
fig_3 = go.Figure(data=p_data_3, layout=layout_3).show()

In [31]:
# Use this to create an output dataframe showing the winning variant at each day of the experiment
# If there are more variants, just copy/paste the elif statement and add the new variants
# If you want to change to conversion rate or revenue-per-send then change the 'P_A_b' and similar to the appropriate column name in df_pivoted
def winning_variant_df(df):
    winning_variant = []
    bayesian_prob = []

    for i in range(len(df)):
        max_prob = max(df.iloc[i]['P_A_b'],df.iloc[i]['P_B_b'],df.iloc[i]['P_C_b'])
        if max_prob == df.iloc[i]['P_A_b']:
            winning_variant.append('A')
            bayesian_prob.append(df.iloc[i]['P_A_b'])

        elif max_prob == df.iloc[i]['P_B_b']:
            winning_variant.append('B')
            bayesian_prob.append(df.iloc[i]['P_B_b'])

        elif max_prob == df.iloc[i]['P_C_b']:
            winning_variant.append('C')
            bayesian_prob.append(df.iloc[i]['P_C_b'])
    
    dict_temp = {'date': df.t, 'winning_variant': winning_variant, 'bayesian_probability': bayesian_prob}
    df_final = pd.DataFrame(dict_temp)
    return df_final

In [32]:
df = winning_variant_df(df_pivoted)
df
#df.to_csv('winning_variant.csv', index=False)

,date,winning_variant,bayesian_probability
0,0,C,1.0000
1,1,C,1.0000
2,2,C,1.0000
3,3,C,1.0000
4,4,C,1.0000
5,5,C,0.6666
6,6,B,1.0000
7,7,C,1.0000
8,8,C,1.0000
9,9,C,1.0000
